<a href="https://colab.research.google.com/github/AliAkbarBadri/topics-for-types/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, GRU
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku 

In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import os

In [ ]:
questions = pd.read_csv("/content/drive/My Drive/topics-for-types/result.csv")


In [ ]:
print(questions.shape)
questions.head()